In [ ]:
# %% [markdown]
# # LLM Hallucination - Complete Guide
# 
# এই notebook এ আমরা শিখবো:
# 1. Hallucination কি এবং কেন হয়
# 2. Hallucination এর types
# 3. Hallucination detect করা
# 4. Hallucination reduce/prevent করা
# 5. Real-world solutions

# %% [markdown]
# ## What is Hallucination? 🤔
# 
# **Hallucination** মানে যখন AI model এমন কিছু generate করে যেটা:
# - ❌ Factually incorrect (ভুল তথ্য)
# - ❌ Doesn't exist (বানানো তথ্য)
# - ❌ Not supported by input/context (context এ নেই)
# - ❌ Logically inconsistent (contradictory)
# 
# ### Real-world Examples:
# 
# 1. **Factual Hallucination:**
#    - Query: "When did Einstein invent the telephone?"
#    - AI: "Einstein invented the telephone in 1876"
#    - Reality: Einstein didn't invent the telephone, it was Alexander Graham Bell
# 
# 2. **Source Hallucination:**
#    - Query: "What does the article say about climate change?"
#    - AI: "The article mentions a 5-degree increase by 2030"
#    - Reality: Article doesn't mention this specific number
# 
# 3. **Confidence Hallucination:**
#    - AI gives completely wrong answer but with 100% confidence
# 
# ### কেন Hallucination হয়?
# 
# 1. **Training Data Issues** - ভুল বা incomplete data
# 2. **Model Limitations** - Model যেটা জানে না সেটা guess করে
# 3. **Ambiguous Queries** - unclear প্রশ্ন
# 4. **Context Missing** - পর্যাপ্ত context না থাকা
# 5. **Over-confidence** - Model too confident in wrong answers

# %% [markdown]
# ## Setup

# %%
# !pip install langgraph langchain-google-genai langchain-community langchain-core python-dotenv sentence-transformers scikit-learn

# %%
import os
from dotenv import load_dotenv
from typing import TypedDict, List, Dict
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

load_dotenv()

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    temperature=0.3,
)

# Initialize embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

print("✅ Setup complete!")

# %% [markdown]
# ## Example 1: Detecting Factual Hallucinations

# %%
def test_hallucination_basic():
    """Test basic hallucination with factual questions"""
    
    questions = [
        "Who invented the telephone?",
        "What is the capital of Mars?",  # Nonsense question
        "When did World War II end?",
        "Who is the current king of United States?",  # US doesn't have a king
    ]
    
    print("="*60)
    print("TESTING FACTUAL HALLUCINATIONS")
    print("="*60)
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. Question: {question}")
        response = llm.invoke([HumanMessage(content=question)])
        print(f"   Answer: {response.content}")
        print("-"*60)

# Run test
test_hallucination_basic()

# %% [markdown]
# ## Example 2: Source-based Hallucination Detection

# %%
# Sample document
SAMPLE_DOCUMENT = """
Tesla Inc. is an American electric vehicle and clean energy company. 
The company was founded in 2003 by Martin Eberhard and Marc Tarpenning.
Elon Musk joined as chairman in 2004 and became CEO in 2008.
Tesla's main products include electric cars like Model S, Model 3, Model X, and Model Y.
The company's headquarters is located in Austin, Texas.
In 2023, Tesla delivered approximately 1.8 million vehicles worldwide.
"""

def check_source_hallucination(context: str, question: str, answer: str) -> dict:
    """Check if answer is grounded in the source"""
    
    # Method 1: Simple keyword matching
    answer_lower = answer.lower()
    context_lower = context.lower()
    
    # Extract key facts from answer
    # For demo, we'll use a simple approach
    
    # Method 2: Use LLM to verify
    verification_prompt = f"""
    Given the following context, verify if the answer is supported by the context.
    
    Context: {context}
    
    Question: {question}
    Answer: {answer}
    
    Respond with JSON:
    {{
        "is_supported": true/false,
        "confidence": 0-100,
        "explanation": "brief explanation",
        "unsupported_claims": ["list of claims not in context"]
    }}
    """
    
    verification = llm.invoke([HumanMessage(content=verification_prompt)])
    
    try:
        result = json.loads(verification.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"is_supported": False, "confidence": 0, "explanation": "Could not parse"}
    
    return result

# Test
print("="*60)
print("TESTING SOURCE-BASED HALLUCINATIONS")
print("="*60)

test_cases = [
    {
        "question": "Who founded Tesla?",
        "answer": "Tesla was founded by Martin Eberhard and Marc Tarpenning in 2003."
    },
    {
        "question": "When did Elon Musk join Tesla?",
        "answer": "Elon Musk joined Tesla in 2004 as chairman."
    },
    {
        "question": "How many cars did Tesla deliver in 2023?",
        "answer": "Tesla delivered approximately 3 million vehicles in 2023."  # WRONG!
    },
    {
        "question": "Where is Tesla's headquarters?",
        "answer": "Tesla's headquarters is in San Francisco, California."  # WRONG!
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n{i}. Question: {test_case['question']}")
    print(f"   Answer: {test_case['answer']}")
    
    result = check_source_hallucination(
        SAMPLE_DOCUMENT,
        test_case['question'],
        test_case['answer']
    )
    
    print(f"   ✓ Supported: {result.get('is_supported', 'Unknown')}")
    print(f"   Confidence: {result.get('confidence', 0)}%")
    print(f"   Explanation: {result.get('explanation', 'N/A')}")
    
    if not result.get('is_supported', False):
        print(f"   ⚠️ Unsupported claims: {result.get('unsupported_claims', [])}")
    
    print("-"*60)

# %% [markdown]
# ## Example 3: RAG with Hallucination Detection

# %%
# Create a sample knowledge base
knowledge_base = [
    "Python was created by Guido van Rossum and first released in 1991.",
    "JavaScript was created by Brendan Eich in 1995 in just 10 days.",
    "Java was developed by James Gosling at Sun Microsystems and released in 1995.",
    "C++ was developed by Bjarne Stroustrup starting in 1979.",
    "Go was designed by Robert Griesemer, Rob Pike, and Ken Thompson at Google.",
]

# Create vector store
docs = [Document(page_content=text) for text in knowledge_base]
vectorstore = Chroma.from_documents(docs, embeddings)

def rag_with_hallucination_check(query: str) -> dict:
    """RAG with built-in hallucination detection"""
    
    # Step 1: Retrieve relevant docs
    retrieved_docs = vectorstore.similarity_search(query, k=2)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    
    print(f"📚 Retrieved Context:")
    print(context)
    print()
    
    # Step 2: Generate answer
    prompt = f"""Based on the following context, answer the question.
    If the answer is not in the context, say "I don't have enough information."
    
    Context: {context}
    
    Question: {query}
    
    Answer:"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    answer = response.content
    
    print(f"🤖 Generated Answer:")
    print(answer)
    print()
    
    # Step 3: Check for hallucination
    if "don't have enough information" in answer.lower() or "not in the context" in answer.lower():
        hallucination_risk = "LOW"
        grounded = True
    else:
        # Verify answer against context
        verification = check_source_hallucination(context, query, answer)
        grounded = verification.get('is_supported', False)
        hallucination_risk = "LOW" if grounded else "HIGH"
    
    return {
        "query": query,
        "context": context,
        "answer": answer,
        "grounded": grounded,
        "hallucination_risk": hallucination_risk
    }

# Test RAG
print("="*60)
print("RAG WITH HALLUCINATION DETECTION")
print("="*60)

test_queries = [
    "Who created Python?",  # In knowledge base
    "When was JavaScript created?",  # In knowledge base
    "What is the capital of France?",  # NOT in knowledge base
    "Who invented Rust?",  # NOT in knowledge base
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    result = rag_with_hallucination_check(query)
    
    print(f"✓ Grounded: {result['grounded']}")
    print(f"⚠️ Hallucination Risk: {result['hallucination_risk']}")
    print()

# %% [markdown]
# ## Example 4: Self-Consistency Check

# %%
def self_consistency_check(question: str, num_samples: int = 3) -> dict:
    """Generate multiple answers and check consistency"""
    
    print(f"Question: {question}\n")
    
    answers = []
    
    # Generate multiple responses
    for i in range(num_samples):
        response = llm.invoke([HumanMessage(content=question)])
        answers.append(response.content)
        print(f"Answer {i+1}: {response.content}")
    
    # Check consistency using LLM
    consistency_prompt = f"""
    Compare these {num_samples} answers to the same question and determine:
    1. Are they consistent with each other?
    2. What are the differences?
    3. Which answer is most likely correct?
    
    Question: {question}
    
    Answers:
    {chr(10).join([f"{i+1}. {ans}" for i, ans in enumerate(answers)])}
    
    Respond with JSON:
    {{
        "consistent": true/false,
        "confidence": 0-100,
        "differences": ["list of differences"],
        "recommended_answer": "the most reliable answer",
        "hallucination_risk": "LOW/MEDIUM/HIGH"
    }}
    """
    
    consistency_result = llm.invoke([HumanMessage(content=consistency_prompt)])
    
    try:
        result = json.loads(consistency_result.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"consistent": False, "confidence": 0}
    
    print(f"\n📊 Consistency Analysis:")
    print(f"   Consistent: {result.get('consistent', 'Unknown')}")
    print(f"   Confidence: {result.get('confidence', 0)}%")
    print(f"   Hallucination Risk: {result.get('hallucination_risk', 'Unknown')}")
    
    return result

# Test self-consistency
print("="*60)
print("SELF-CONSISTENCY CHECK")
print("="*60)

test_questions = [
    "What is 25 * 47?",  # Factual, should be consistent
    "Who was the 100th president of the United States?",  # Nonsense, may be inconsistent
]

for question in test_questions:
    print(f"\n{'='*60}")
    result = self_consistency_check(question, num_samples=3)
    print()

# %% [markdown]
# ## Example 5: Chain-of-Thought with Verification

# %%
def chain_of_thought_with_verification(question: str) -> dict:
    """Use CoT and then verify the reasoning"""
    
    # Step 1: Get answer with Chain of Thought
    cot_prompt = f"""
    Answer the following question step by step. Show your reasoning.
    
    Question: {question}
    
    Think through this carefully:
    1. What information do I need?
    2. What do I know?
    3. What is my reasoning?
    4. What is my final answer?
    """
    
    response = llm.invoke([HumanMessage(content=cot_prompt)])
    reasoning = response.content
    
    print(f"🧠 Chain of Thought Reasoning:")
    print(reasoning)
    print()
    
    # Step 2: Verify the reasoning
    verification_prompt = f"""
    Verify the following reasoning and answer. Point out any logical errors,
    unsupported claims, or potential hallucinations.
    
    Question: {question}
    
    Reasoning and Answer:
    {reasoning}
    
    Provide verification in JSON:
    {{
        "logical_errors": ["list of errors"],
        "unsupported_claims": ["claims without evidence"],
        "confidence_score": 0-100,
        "hallucination_detected": true/false,
        "corrected_answer": "if hallucination found, provide correct answer"
    }}
    """
    
    verification = llm.invoke([HumanMessage(content=verification_prompt)])
    
    try:
        result = json.loads(verification.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"hallucination_detected": False}
    
    print(f"✓ Verification Results:")
    print(json.dumps(result, indent=2))
    
    return {
        "question": question,
        "reasoning": reasoning,
        "verification": result
    }

# Test CoT with verification
print("="*60)
print("CHAIN-OF-THOUGHT WITH VERIFICATION")
print("="*60)

test_questions = [
    "If a train travels 120 miles in 2 hours, what is its average speed?",
    "Who won the World Cup in 2050?",  # Future event
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print('='*60)
    result = chain_of_thought_with_verification(question)
    print()

# %% [markdown]
# ## Example 6: Attribution-based Hallucination Detection

# %%
def attribution_check(context: str, claim: str) -> dict:
    """Check if a specific claim can be attributed to the context"""
    
    # Use embeddings to check similarity
    context_embedding = embeddings.embed_query(context)
    claim_embedding = embeddings.embed_query(claim)
    
    # Calculate cosine similarity
    similarity = cosine_similarity(
        [context_embedding],
        [claim_embedding]
    )[0][0]
    
    # Use LLM for detailed attribution
    attribution_prompt = f"""
    Context: {context}
    
    Claim: {claim}
    
    Can this claim be directly attributed to the context?
    If yes, quote the relevant part.
    If no, explain why.
    
    Respond with JSON:
    {{
        "attributed": true/false,
        "confidence": 0-100,
        "supporting_quote": "exact quote if found, else null",
        "explanation": "brief explanation"
    }}
    """
    
    response = llm.invoke([HumanMessage(content=attribution_prompt)])
    
    try:
        result = json.loads(response.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"attributed": False, "confidence": 0}
    
    result['embedding_similarity'] = float(similarity)
    
    return result

# Test attribution
print("="*60)
print("ATTRIBUTION-BASED HALLUCINATION DETECTION")
print("="*60)

context = """
Apple Inc. announced its Q4 2023 earnings today. 
The company reported revenue of $89.5 billion, up 1% year over year.
iPhone sales contributed $43.8 billion to the revenue.
CEO Tim Cook stated that the company is excited about the Vision Pro launch.
"""

claims = [
    "Apple's Q4 2023 revenue was $89.5 billion",  # TRUE
    "iPhone sales were $43.8 billion",  # TRUE
    "Apple's revenue decreased by 5% year over year",  # FALSE
    "Apple announced a new MacBook Pro",  # NOT MENTIONED
]

for claim in claims:
    print(f"\nClaim: {claim}")
    result = attribution_check(context, claim)
    
    print(f"  ✓ Attributed: {result['attributed']}")
    print(f"  Confidence: {result['confidence']}%")
    print(f"  Embedding Similarity: {result['embedding_similarity']:.3f}")
    
    if result.get('supporting_quote'):
        print(f"  Quote: \"{result['supporting_quote']}\"")
    else:
        print(f"  Explanation: {result['explanation']}")
    
    print("-"*60)

# %% [markdown]
# ## Example 7: Confidence Calibration

# %%
def confidence_calibration(question: str) -> dict:
    """Ask model to provide confidence score with answer"""
    
    prompt = f"""
    Answer the following question and provide a confidence score (0-100).
    If you're not sure, be honest about it.
    
    Question: {question}
    
    Respond with JSON:
    {{
        "answer": "your answer",
        "confidence": 0-100,
        "reasoning": "why this confidence level",
        "sources_of_uncertainty": ["what makes you uncertain"]
    }}
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    try:
        result = json.loads(response.content.strip().replace("```json", "").replace("```", ""))
    except:
        result = {"answer": response.content, "confidence": 50}
    
    return result

# Test confidence calibration
print("="*60)
print("CONFIDENCE CALIBRATION")
print("="*60)

test_questions = [
    "What is the capital of France?",  # Easy, high confidence
    "What is the population of Iceland?",  # Harder, medium confidence
    "What is the GDP of Bhutan?",  # Hard, low confidence
    "Who will win the next World Cup?",  # Impossible, should be very low
]

for question in test_questions:
    print(f"\nQuestion: {question}")
    result = confidence_calibration(question)
    
    print(f"  Answer: {result['answer']}")
    print(f"  Confidence: {result['confidence']}%")
    print(f"  Reasoning: {result.get('reasoning', 'N/A')}")
    
    if result['confidence'] < 50:
        print(f"  ⚠️ LOW CONFIDENCE - High hallucination risk!")
    
    print("-"*60)

# %% [markdown]
# ## Best Practices to Reduce Hallucinations
# 
# ### 1️⃣ Prompt Engineering
# ```python
# # ❌ Bad Prompt
# "Tell me about XYZ"
# 
# # ✅ Good Prompt
# "Based on the provided context, tell me about XYZ. 
#  If the information is not in the context, say 'I don't know'."
# ```
# 
# ### 2️⃣ RAG (Retrieval-Augmented Generation)
# - Always provide context
# - Use vector databases
# - Verify answers against sources
# 
# ### 3️⃣ Self-Consistency
# - Generate multiple answers
# - Check for consistency
# - Flag inconsistencies
# 
# ### 4️⃣ Chain-of-Thought
# - Ask for step-by-step reasoning
# - Easier to spot logical errors
# - Verify each step
# 
# ### 5️⃣ Attribution
# - Require citations
# - Check if claims match sources
# - Use embedding similarity
# 
# ### 6️⃣ Confidence Scores
# - Ask model for confidence
# - Flag low-confidence answers
# - Human review for uncertain answers
# 
# ### 7️⃣ Human-in-the-Loop
# - Critical decisions need human verification
# - Use HITL for high-stakes applications
# 
# ### 8️⃣ Post-Processing
# - Verify facts against knowledge base
# - Use fact-checking APIs
# - Cross-reference multiple sources

# %% [markdown]
# ## Complete Anti-Hallucination System

# %%
class AntiHallucinationSystem:
    """Complete system to detect and prevent hallucinations"""
    
    def __init__(self, llm, embeddings, knowledge_base=None):
        self.llm = llm
        self.embeddings = embeddings
        self.knowledge_base = knowledge_base
        
    def generate_with_checks(self, query: str, context: str = None) -> dict:
        """Generate answer with multiple hallucination checks"""
        
        results = {
            "query": query,
            "checks_performed": [],
            "warnings": [],
            "final_answer": None,
            "confidence": 0,
            "safe_to_use": False
        }
        
        # Check 1: Generate answer with context
        if context:
            prompt = f"""Based on this context, answer the question.
            If not in context, say "I don't have enough information."
            
            Context: {context}
            Question: {query}
            """
        else:
            prompt = query
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        answer = response.content
        
        results['initial_answer'] = answer
        results['checks_performed'].append("✓ Initial generation")
        
        # Check 2: Self-consistency
        answers = [answer]
        for _ in range(2):
            resp = self.llm.invoke([HumanMessage(content=prompt)])
            answers.append(resp.content)
        
        # Simple consistency check
        all_similar = len(set(answers)) <= 2  # Allow minor variations
        results['checks_performed'].append(f"✓ Self-consistency: {'PASS' if all_similar else 'FAIL'}")
        
        if not all_similar:
            results['warnings'].append("⚠️ Inconsistent answers detected")
        
        # Check 3: Source attribution (if context provided)
        if context:
            attribution = attribution_check(context, answer)
            results['checks_performed'].append(f"✓ Attribution: {'PASS' if attribution['attributed'] else 'FAIL'}")
            
            if not attribution['attributed']:
                results['warnings'].append("⚠️ Answer not well-supported by context")
        
        # Check 4: Confidence calibration
        conf_result = confidence_calibration(query)
        confidence = conf_result.get('confidence', 50)
        results['confidence'] = confidence
        results['checks_performed'].append(f"✓ Confidence: {confidence}%")
        
        if confidence < 60:
            results['warnings'].append(f"⚠️ Low confidence ({confidence}%)")
        
        # Final decision
        if len(results['warnings']) == 0 and confidence >= 70:
            results['safe_to_use'] = True
            results['final_answer'] = answer
        elif len(results['warnings']) <= 1 and confidence >= 60:
            results['safe_to_use'] = True
            results['final_answer'] = f"⚠️ {answer}\n(Note: Answer has medium confidence)"
        else:
            results['safe_to_use'] = False
            results['final_answer'] = "I cannot provide a reliable answer. Please verify with other sources."
        
        return results

# Test the complete system
print("="*60)
print("COMPLETE ANTI-HALLUCINATION SYSTEM")
print("="*60)

system = AntiHallucinationSystem(llm, embeddings)

context = """
The Moon is Earth's only natural satellite. It orbits Earth at an average
distance of 384,400 km. The Moon's diameter is 3,474 km, about one-quarter
of Earth's diameter. It takes 27.3 days to orbit Earth.
"""

test_cases = [
    ("What is the Moon's diameter?", context),
    ("How long does it take the Moon to orbit Earth?", context),
    ("What is the temperature on the Moon?", context),  # Not in context
]

for query, ctx in test_cases:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print('='*60)
    
    result = system.generate_with_checks(query, ctx)
    
    print("\nChecks Performed:")
    for check in result['checks_performed']:
        print(f"  {check}")
    
    if result['warnings']:
        print("\n⚠️ Warnings:")
        for warning in result['warnings']:
            print(f"  {warning}")
    
    print(f"\nConfidence: {result['confidence']}%")
    print(f"Safe to use: {result['safe_to_use']}")
    print(f"\nFinal Answer: {result['final_answer']}")

# %% [markdown]
# ## Summary & Takeaways
# 
# ### 🎓 What We Learned:
# 
# 1. **Types of Hallucinations:**
#    - Factual hallucinations
#    - Source hallucinations
#    - Confidence hallucinations
# 
# 2. **Detection Methods:**
#    - Source verification
#    - Self-consistency checks
#    - Attribution checking
#    - Confidence calibration
#    - Chain-of-thought verification
# 
# 3. **Prevention Strategies:**
#    - Use RAG with proper context
#    - Implement multiple verification layers
#    - Request confidence scores
#    - Use self-consistency
#    - Employ human-in-the-loop
# 
# ### 💡 Key Principles:
# 
# ✅ **Always provide context** - Don't let the model guess
# ✅ **Verify against sources** - Check if answer is grounded
# ✅ **Use multiple checks** - Combine different methods
# ✅ **Request confidence** - Know when to trust the answer
# ✅ **Human review for critical** - Don't automate everything
# 
# ### 🚫 Never Do:
# 
# ❌ Trust LLM output blindly for critical applications
# ❌ Use LLMs without context for factual questions
# ❌ Ignore low confidence scores
# ❌ Skip verification steps to save time
# 
# ### 🎯 When to Use What:
# 
# | Use Case | Best Method |
# |----------|-------------|
# | Factual Q&A | RAG + Attribution |
# | Creative Writing | Self-Consistency |
# | Critical Decisions | Full System + HITL |
# | Simple Queries | Confidence Calibration |
# | Research | Multiple Sources + Verification |

# %%
print("✅ Hallucination Tutorial Complete!")
print("\n📚 You now know:")
print("  - What hallucinations are and why they occur")
print("  - How to detect different types of hallucinations")
print("  - Multiple methods to reduce hallucinations")
print("  - How to build a complete anti-hallucination system")
print("\n🎉 You're ready to build reliable AI systems!")